In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
import re
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.neural_network import MLPClassifier as cnn

In [2]:
data_path1 = '../dataset/Tweets-airline-sentiment.csv'
#data_path2 = '../dataset/labeledTrainData_head12000.tsv'

In [3]:
data = pd.read_csv(data_path1)

In [4]:
#data.head()

In [5]:
text = data['text']

In [6]:
label = data['airline_sentiment']

In [7]:
label_tags = label.unique()
print(label_tags)

['neutral' 'positive' 'negative']


In [8]:
#replace text label with one-hot-labels


In [9]:
new_label = []
for l in label:
    if l == label_tags[0]:
        #new_label.append(np.array([0,0,1]))
        new_label.append(0)
    elif l == label_tags[1]:
        #new_label.append(np.array([0,1,0]))
        new_label.append(1)
    else:
        #new_label.append(np.array([1,0,0]))
        new_label.append(2)

In [10]:
new_label = np.asarray(new_label)

In [11]:
#get rid of '@airline_company_name
new_text = []

In [12]:
for t in text:
    new_text.append(re.sub('^@\w+ *','', t))

In [13]:
new_text = np.asarray(new_text)

In [14]:
new_text.shape, new_label.shape

((14640,), (14640,))

# Split

In [15]:
def split(x, y, n_split, test_train_split = 0.2):
# x: traning test, in array
#y: label
#n_split: number of groups divided from x and y
    if x.shape[0] != y.shape[0]:
        raise('samples and their labels\'number mismatched')
    else:
        instance_num = x.shape[0]
        train_num = int(instance_num*(1-test_train_split))
        permulated_arr = np.random.permutation(instance_num)
        train_arr, test_arr = permulated_arr[:train_num], permulated_arr[train_num:]
        train_each_num = int(train_num/n_split)
        test_each_num = int((instance_num-train_num)/n_split)
        for i in range(n_split):
            if i == n_split-1:
                #train index and test index
                yield (train_arr[i*train_each_num:], test_arr[i*test_each_num:])
            else:
                yield (train_arr[i*train_each_num:(i+1)*train_each_num], test_arr[i*test_each_num:(i+1)*test_each_num])

In [87]:
for train, test in split(uni, new_label,5):
    print(train.shape)
    print(test.shape)

(2342,)
(585,)
(2342,)
(585,)
(2342,)
(585,)
(2342,)
(585,)
(2344,)
(588,)


# model

In [16]:
NB = MultinomialNB()
pc = Perceptron()
svm = LinearSVC()
lr = LogisticRegression()
random_forest  = rf()

CNN = cnn()

In [36]:
KNN = knn(n_neighbors=6)

# Unigram

In [31]:
UniVec = CountVectorizer(max_features = 500, ngram_range = (1,1))
uni = UniVec.fit_transform(new_text)

In [32]:
uni = np.asarray(uni.todense())

In [33]:
uni.shape

(14640, 500)

In [34]:
skf = StratifiedKFold(n_splits=5)


In [21]:
for clf in [NB, pc, svm, lr]:
    acc = []
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.7455674578775445


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.7357973167295876
0.7697455739085189
0.7799217947218307


In [23]:
#use my split()
'''
for clf in [ NB,pc, svm, lr]:
    acc = []
    for train_index, test_index in split(uni, new_label,5):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())
'''

'\nfor clf in [ NB,pc, svm, lr]:\n    acc = []\n    for train_index, test_index in split(uni, new_label,5):\n        x_train,x_test = uni[train_index], uni[test_index]\n        y_train, y_test = new_label[train_index], new_label[test_index]\n        clf.fit(x_train, y_train)\n        acc.append(clf.score(x_test, y_test))\n    acc = np.asarray(acc)\n    print(acc.mean())\n'

In [37]:
for clf in [KNN, random_forest]:
    acc = []
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.4887411030651731
0.7307385410044167


In [ ]:
for clf in [CNN]:
    acc = []
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

# Bigram

In [24]:
BiVec = CountVectorizer(max_features = 1000, ngram_range = (2,2))
Bi = BiVec.fit_transform(new_text)
skf = StratifiedKFold(n_splits=5)

In [25]:
for clf in [NB, pc, svm, lr, KNN, random_forest]:
    acc = []
    for train_index, test_index in skf.split(Bi, new_label):
        x_train,x_test = Bi[train_index], Bi[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6807406780510499
0.6459729581957722


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6883211764972066
0.7031416470449299
0.5224757379084302
0.6462472329414705


In [26]:
for clf in [CNN]:
    acc = []
    for train_index, test_index in skf.split(Bi, new_label):
        x_train,x_test = Bi[train_index], Bi[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6692664559871363


# Uni&Bigram

In [27]:
MixVec = CountVectorizer(max_features = 1000, ngram_range = (1,2))
Mix = BiVec.fit_transform(new_text)
skf = StratifiedKFold(n_splits=5)

In [28]:
for clf in [NB, pc, svm, lr, KNN, random_forest]:
    acc = []
    for train_index, test_index in skf.split(Mix, new_label):
        x_train,x_test = Mix[train_index], Mix[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6807406780510499
0.6459729581957722


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6883211764972066
0.7031416470449299
0.5224757379084302
0.649184321720502


In [29]:
for clf in [CNN]:
    acc = []
    for train_index, test_index in skf.split(Mix, new_label):
        x_train,x_test = Mix[train_index], Mix[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6656449069229424
